# Convolutional Autoencoder

Yeah, I'm apparently actually doing this.

Structure is roughly as follows.

- Convolutional layer
- Symmetric(!!!) activation layer
- Max pooling layer
- Dropout layer
- Flatten layer
- Dense layer
- Symmetric activation layer
- Unflatten layer
- Unpool layer
- Deconvolutional layer
- Symmetric activation layer

Inspiration from https://github.com/mikesj-public/convolutional_autoencoder/blob/master/mnist_conv_autoencode.py and https://github.com/nanopony/keras-convautoencoder/blob/master/conv_autoencoder.py.

Let's load up the dataset I put up on Zenodo.

In [1]:
import h5py, numpy

with h5py.File('../data/dataset.h5', 'r') as f:
    features = f['features'][:, :32 * 32]

Now we'll build an autoencoder...

In [15]:
import keras, \
        keras.layers, \
        keras.layers.core as core, \
        keras.layers.convolutional as conv, \
        keras.models as models
from keras import backend as K
from keras.engine.topology import Layer

n_filters = 32
conv_size = 5
pool_size = 2
dropout = 0.25
patch_size = 32
conv_out_size = (patch_size - conv_size) // pool_size + 1
hidden = n_filters * conv_out_size

In [21]:
class Sum(Layer):
    def __init__(self, **kwargs):
        super(Sum, self).__init__(**kwargs)

    def build(self, input_shape):
        self.trainable_weights = []

    def call(self, x, mask=None):
        return x.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0],) + input_shape[2:]

In [23]:
model = models.Sequential()

# Encoder.
model.add(conv.Convolution2D(n_filters, conv_size, conv_size,
                             border_mode='valid',
                             input_shape=(1, patch_size, patch_size)))
model.add(core.Activation('tanh'))
model.add(conv.MaxPooling2D(pool_size=(pool_size, pool_size)))
model.add(core.Dropout(dropout))
model.add(core.Flatten())

# Dense.
# model.add(core.Dense(n_filters * conv_out_size * conv_out_size))
# model.add(core.Activation('tanh'))

# Decoder.
model.add(core.Reshape((n_filters, conv_out_size, conv_out_size)))
model.add(conv.UpSampling2D(size=(pool_size, pool_size)))
model.add(conv.ZeroPadding2D(padding=(conv_size - 1, conv_size - 1)))
deconv = conv.Convolution2D(n_filters, conv_size, conv_size,
                            border_mode='valid')
model.add(deconv)
model.add(Sum())

model.compile(loss='mse', optimizer='adadelta')

In [ ]:
images = features.reshape((-1, patch_size, patch_size))
model.fit(images[:1000].reshape((-1, 1, patch_size, patch_size)), images[:1000])

Epoch 1/10
 480/1000 [=============>................] - ETA: 25s - loss: 0.5349